In [1]:
import numpy as np
import math
import pandas as pd
import re
from collections import Counter
import os

In [2]:
def buildUnigramModel(Text):
    kata = []
    for i in Text:
        # lower case setiap kata
        i = i.strip().lower()
        # pisah kata
        kata += re.split(r'\W+', i)
        
    count = dict(Counter(kata))
    total = len(kata)
    
    prob = {}
    for key, value in count.items():
        prob[key] = value / total 
    return prob

In [3]:
def buildBigramModel(Text):
    kalimat, kata = [], []

    for i in Text:
        # split per kalimat
        kalimat += re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<!vs.)(?<=\.|\?)\s", i)

    for i in range (len(kalimat)):
        # lower case setiap kata
        kalimat[i] = kalimat[i].strip().lower()
        # pemisahan setiap kata sekalian tambah token
        kata += ['<s>'] + re.split(r'\W+', kalimat[i]) + ['</s>']

    # hapus isi kosong
    kata = [x for x in kata if x != '']
    kata_unigram = kata.copy()

    count_unigram = dict(Counter(kata_unigram))

    bigram = [(kata[i], kata[i+1]) for i in range(len(kata)-1)]
    bigram.remove(('</s>', '<s>'))

    count_bigram = dict(Counter(bigram))

    prob = {}
    for key, value in count_bigram.items():
        prob[key] = value / count_unigram[key[0]]
    return prob

In [19]:
def nextBestWord(bigramModel, currentWord):
    prob = []
    for key, value in bigramModel.items():
        # kalau kata nya sama, masukin list
        if key[0] == currentWord:
            prob.append((key[1], value))
    # sort by kemungkinan munculnya
    prob = sorted(prob, key=lambda x: x[1], reverse=True)
    # return nomor 1
    return prob[0]

In [5]:
def nextTenBestWords(bigramModel, currentWord):
    prob = []
    for key, value in bigramModel.items():
        if key[0] == currentWord:
            prob.append((key[1], value))
    prob = sorted(prob, key=lambda x: x[1], reverse=True)
    return prob[:10]

In [18]:
def generateSentence(bigramModel, length):
    threshold = 0.3
    current = "<s>"
    sentence = ""
    for i in range (length):
        if np.random.uniform(0,1) < threshold:
            current = nextBestWord(bigramModel, current)[0]
            sentence += current + " "
        else:
            top10 = nextTenBestWords(bigramModel, current)
            current = top10[np.random.randint(0,len(top10))][0]
            sentence += current + " "
    return sentence

In [10]:
data = pd.read_csv('text.csv')
print("TUGAS 1. TAMPILKAN 5 BARIS PERTAMA DARI DATASET")
print()
print("HASIL : ")
print(data.head())

TUGAS 1. TAMPILKAN 5 BARIS PERTAMA DARI DATASET

HASIL : 
                                                text
0  Oh, how the headlines blared:\nChatbots were T...
1  If you’ve ever found yourself looking up the s...
2  Machine learning is increasingly moving from h...
3  If your understanding of A.I. and Machine Lear...
4  Want to learn about applied Artificial Intelli...


In [11]:
print("TUGAS 2. BUAT MODEL UNIGRAM")
print()
print("HASIL : ")
print(buildUnigramModel(data['text']))

TUGAS 2. BUAT MODEL UNIGRAM

HASIL : 
{'oh': 2.5356512566687627e-05, 'how': 0.003270990121102704, 'the': 0.04595107207335132, 'headlines': 2.0285210053350103e-05, 'blared': 1.0142605026675052e-05, 'chatbots': 0.00020285210053350101, 'were': 0.0013388238635211067, 'next': 0.0008316936121873542, 'big': 0.000791123192080654, 'thing': 0.0004209181086070146, 'our': 0.0047720956650506114, 'hopes': 2.0285210053350103e-05, 'sky': 2.0285210053350103e-05, 'high': 0.0004969876463070775, 'bright': 2.5356512566687627e-05, 'eyed': 1.0142605026675052e-05, 'and': 0.022886788242692253, 'bushy': 1.0142605026675052e-05, 'tailed': 1.0142605026675052e-05, 'industry': 0.00019270949550682597, 'was': 0.002119804450575086, 'ripe': 1.0142605026675052e-05, 'for': 0.010218674564375115, 'a': 0.027065541513682373, 'new': 0.0015011055439479075, 'era': 0.00011156865529342556, 'of': 0.026791691177962148, 'innovation': 8.114084021340041e-05, 'it': 0.010720733513195529, 'time': 0.0019727366776882975, 'to': 0.02996632655

In [12]:
print("TUGAS 3. BUAT MODEL BIGRAM")
print()
print("HASIL : ")
bigramModel = buildBigramModel(data['text'])
print(bigramModel)

TUGAS 3. BUAT MODEL BIGRAM

HASIL : 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
print("TUGAS 4. MENAMPILKAN NEXT BEST WORD")
print()
print("HASIL : ")
print("of -> ",nextBestWord(bigramModel,"of"))
print("update -> ",nextBestWord(bigramModel,"update"))
print("hopes -> ",nextBestWord(bigramModel,"hopes"))

TUGAS 4. MENAMPILKAN NEXT BEST WORD

HASIL : 
of ->  ('the', 0.1938292636759417)
update ->  ('this', 0.24444444444444444)
hopes ->  ('were', 0.5)


In [14]:
print("TUGAS 5. TOP 10 BEST NEXT WORD")
print()
print("HASIL : ")
print("of -> ",nextTenBestWords(bigramModel,"of"))
print("update -> ",nextTenBestWords(bigramModel,"update"))
print("hopes -> ",nextTenBestWords(bigramModel,"hopes"))

TUGAS 5. TOP 10 BEST NEXT WORD

HASIL : 
of ->  [('the', 0.1938292636759417), ('a', 0.045618020064357376), ('our', 0.022335794056407343), ('this', 0.022146507666098807), ('data', 0.016846488737459777), ('these', 0.015142911224682946), ('machine', 0.011357183418512209), ('what', 0.008517887563884156), ('each', 0.008139314783267083), ('them', 0.00776074200265001)]
update ->  [('this', 0.24444444444444444), ('the', 0.13333333333333333), ('our', 0.1111111111111111), ('1', 0.06666666666666667), ('2', 0.06666666666666667), ('its', 0.06666666666666667), ('it', 0.044444444444444446), ('4', 0.044444444444444446), ('recommendations', 0.022222222222222223), ('cluster', 0.022222222222222223)]
hopes ->  [('were', 0.5), ('to', 0.25), ('of', 0.25)]


In [17]:
print("TUGAS 6. GENERATE KALIMAT")
print()
n = int(input("Panjang Kalimat : "))
print("HASIL : ")
print(generateSentence(bigramModel, n))

TUGAS 6. GENERATE KALIMAT

Panjang Kalimat : 10
HASIL : 
0
Current:  <s>
Sentence:  
1
Current:  it
Sentence:  it 
2
Current:  s
Sentence:  it s 
3
Current:  machine
Sentence:  it s machine 
4
Current:  intelligence
Sentence:  it s machine intelligence 
5
Current:  to
Sentence:  it s machine intelligence to 
6
Current:  the
Sentence:  it s machine intelligence to the 
7
Current:  same
Sentence:  it s machine intelligence to the same 
8
Current:  time
Sentence:  it s machine intelligence to the same time 
9
Current:  as
Sentence:  it s machine intelligence to the same time as 
it s machine intelligence to the same time as a 
